In [1]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import ast


/kaggle/input/articles/df_dict_test.csv


**Роман мне скинул чанки, сделанные парсером, я очень доволен!!**

In [2]:
df = pd.read_csv('/kaggle/input/articles/df_dict_test.csv')
df

,id,title,abstract,dict_test
0,1602.04402,balanced truncation of linear time-invariant s...,this paper discusses model order reduction of ...,{'I. INTRODUCTION AND PROBLEM FORMULATIONS': '...
1,1611.01462,tying word vectors and word classifiers: a los...,recurrent neural networks have been very succe...,"{'INTRODUCTION': ""Neural network models have r..."
2,1611.04496,multi-view recurrent neural acoustic word embe...,recent work has begun exploring neural acousti...,{'INTRODUCTION': 'Word embeddings-continuous-v...
3,1808.00560,compressible spectral mixture kernels with spa...,spectral mixture (sm) kernels comprise a power...,"{'Introduction': ""Gaussian processes (GPs) con..."
4,2111.00405,limitations of the macaulay matrix approach fo...,recently chen and gao~\cite{chengao2017} propo...,{'Introduction': 'Solving systems of multivari...
...,...,...,...,...
95,2307.14341,virtual mirrors: non-line-of-sight imaging bey...,non-line-of-sight (nlos) imaging methods are c...,{'Computed image of': 'T-shaped object from a ...
96,2307.14354,learned gridification for efficient point clou...,neural operations that rely on neighborhood in...,{'Introduction': 'Point clouds provide sparse ...
97,2307.14362,learnable wavelet neural networks for cosmolog...,convolutional neural networks (cnns) have been...,{'Introduction': 'The process of extracting in...
98,2307.14392,human-centric scene understanding for 3d large...,human-centric scene understanding is significa...,{'Introduction': 'Human-centric scene understa...


*Пример для одной статьи:*

In [40]:
data_dict = ast.literal_eval(df['dict_test'].iloc[0])
print(*data_dict.keys(),sep='\n')
print(f'Количество чанков: {len(data_dict.keys())}',end='\n\n')

for key, value in data_dict.items():
    print(key,end='\n\n')
    print(value,end='\n\n')



I. INTRODUCTION AND PROBLEM FORMULATIONS
DRAFT
II. FUNDAMENTAL TOOL
IV. FREQUENCY-DEPENDENT BALANCED TRUNCATION OVER KNOWN FREQUENCY-INTERVALS
Theorem 4.4 (Interval-type Frequency-dependent Balanced Truncation):
, n
V. EXAMPLES
Indexes computation formula
Berlin/Heidelberg, Germany
Количество чанков: 9

I. INTRODUCTION AND PROBLEM FORMULATIONS

We study model order reduction for linear time-invariant continuous-time systems where A ∈ C n×n , B ∈ C n×m , C ∈ C p×n , D ∈ C p×m , x(t) ∈ C n is the state vector, u(t) ∈ C m is the input signal, y(t) ∈ C p is the output signal. Modeling of complex physical processes often leads to large order n. The corresponding high storage requirements and expensive computations make it very difficult to simulate, optimize or even design such large scale systems - . In this case model order reduction (MOR) plays an important role. It consists in approximating the system (1) by a reduced-order system: G r (ω) : 1 Max Planck Institute for Dynamics of Compl

In [9]:
!pip install -U bitsandbytes accelerate transformers -q
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-4B-Instruct-2507")
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen3-4B-Instruct-2507",
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

torch.cuda.empty_cache()
!pip install langchain==0.0.208  -q
from langchain.text_splitter import RecursiveCharacterTextSplitter
!pip install rouge-score evaluate -q
from rouge_score import rouge_scorer
import evaluate
rouge = evaluate.load('rouge')

!pip install bert-score -q
from bert_score import BERTScorer

scorer = BERTScorer(lang="en", model_type="bert-base-multilingual-cased")

!pip install longdocfactscore -q
import nltk
nltk.download('punkt_tab')
nltk.download('punkt')
from longdocfactscore.ldfacts import LongDocFACTScore
ldfacts_scorer = LongDocFACTScore(device=device)
from tqdm.notebook import tqdm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 16.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 27.1 MB/s eta 0:00:0000:0100:01
cuda


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

Функция для того чтобы отправить инструкцию модели

In [31]:
def qwen(article, prompt, max_tokens):
    torch.cuda.empty_cache()
    final_prompt = f"{prompt}:\n\n{article}"
    messages = [{"role": "user", "content": final_prompt}]
    model.eval()
    with torch.no_grad():
        inputs = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            tokenize=True,
            return_dict=True,
            return_tensors="pt",
            truncation=False
        ).to(model.device)
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            do_sample=False
        )
    final_summary = tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True)
    return final_summary

Функция для обработки чанков статьи, встроенный text_splitter можно отключить

In [48]:
def summarize(article, prompt_0, prompt_1,chunk_size=1500, max_tokens_0=700, max_tokens_1=700, text_splitter = True):
    torch.cuda.empty_cache()
    if text_splitter:
        text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
            tokenizer=tokenizer,
            chunk_size=chunk_size,
            chunk_overlap=chunk_size//20,
            separators=["\n\n", "\n", ". "],
            keep_separator=True
        )
        article_chunks = text_splitter.split_text(article)
    else:
        article_chunks = article
    summaries = []
    for i, chunk in tqdm(enumerate(article_chunks), total=len(article_chunks)):
        try:
            chunk_summary = qwen(article=chunk, prompt=prompt_0, max_tokens=max_tokens_0)
            print(f"\nSummary chunk number {i}")
            print(chunk_summary)
        except torch.cuda.OutOfMemoryError as e:
            print(f"⚠️ Ошибка нехватки памяти CUDA, слишком длинный чанк: {str(e)[:100]}...")
            break

        summaries.append(chunk_summary)
        torch.cuda.empty_cache()
    combined_summary = "\n".join(summaries)
    
    try:
        final_summary = qwen(article=combined_summary, prompt=prompt_1, max_tokens=max_tokens_1)
    except torch.cuda.OutOfMemoryError as e:
        print(f"⚠️ Ошибка нехватки памяти CUDA при итоговой суммаризации: {str(e)[:100]}...")
    return final_summary

In [ ]:
rouge_list = []
bert_list = []
ldfacts_list = []
summaries_list = []

def to_full_text(data_dict):
    text = ''
    for key, value in data_dict.items(): 
        text += f"{key}\n\n{value}\n\n"
    return text


for i in range(5):
    print(f"\n=== Статья {i} ===")
    #извлекаем чанки, абстракт, названия разделов
    article_dict = ast.literal_eval(df['dict_test'].iloc[i]) #словарь для i-той статьи 
    chunked_article = list(article_dict.values()) #список чанков
    chapters_list = list(article_dict.keys()) #список глав
    print('Список разделов')
    display(pd.DataFrame(chapters_list))
    abstract = df['abstract'].iloc[i] 
    article = to_full_text(article_dict) #полный текст статьи с ключами и значениями словаря
    
    print(f"Токенов в статье: {len(tokenizer.encode('\n'.join(chunked_article)))}")
    prompt_0 = "You're a science editor. Briefly summarize this fragment of the scientific text in original language. Do not add information that is not in the source texts"
    prompt_1 = "You're a science editor. Based on the following summaries of the parts of the article, create a single, coherent and concise summary of the entire scientific article in original language, highlighting the common goal, methods, key results and conclusion. Do not add information that is not in the source texts"
    
    summary_full = summarize(chunked_article, prompt_0 = prompt_0, prompt_1 = prompt_1, max_tokens_0 = 500, max_tokens_1 = 700, text_splitter = False)
    print(f'\nSummary article number {i}')
    print(summary_full)
    summaries_list.append({
        'article_id': i,
        'original_text': article,
        'abstract': abstract,
        'summary': summary_full
    })

    results = rouge.compute(
    predictions=[summary_full],
    references=[article],
    use_stemmer=True
    )
    print("ROUGE Metrics:")

    rouge_dict = {}
    for key, value in results.items():
        print(f"{key}: {value:.4f}")
        rouge_dict[key] = value
    rouge_list.append(rouge_dict)

    _, _, F1 = scorer.score([summary_full], [abstract])
    bert_f1 = F1.item()
    print(f"\nBERTScore: F1 = {bert_f1:.4f}")
    bert_list.append(bert_f1)

    ldfacts_sum = ldfacts_scorer.score_src_hyp_long([article], [summary_full])
    ldfacts_abs = ldfacts_scorer.score_src_hyp_long([article], [abstract])
    print(f"LongDocFACTScore for sum: {ldfacts_sum[0]}")
    print(f"LongDocFACTScore for abstract: {ldfacts_abs[0]}")
    ldfacts_list.append({
        'for_summary': ldfacts_sum[0],
        'for_abstract': ldfacts_abs[0]
    })


=== Статья 0 ===
Список разделов


,0
0,I. INTRODUCTION AND PROBLEM FORMULATIONS
1,DRAFT
2,II. FUNDAMENTAL TOOL
3,IV. FREQUENCY-DEPENDENT BALANCED TRUNCATION OV...
4,Theorem 4.4 (Interval-type Frequency-dependent...
5,", n"
6,V. EXAMPLES
7,Indexes computation formula
8,"Berlin/Heidelberg, Germany"


Токенов в статье: 6903


  0%|          | 0/9 [00:00<?, ?it/s]


Summary chunk number 0
We study model order reduction for linear time-invariant continuous-time systems with state vector \( x(t) \in \mathbb{C}^n \), input \( u(t) \in \mathbb{C}^m \), and output \( y(t) \in \mathbb{C}^p \), described by matrices \( A \in \mathbb{C}^{n\times n} \), \( B \in \mathbb{C}^{n\times m} \), \( C \in \mathbb{C}^{p\times n} \), \( D \in \mathbb{C}^{p\times m} \). Large system orders \( n \) lead to high computational and storage costs, making simulation, optimization, and design challenging. Model order reduction (MOR) is thus essential to approximate the system by a reduced-order model \( G_r(j\omega) \) with \( r < n \). Balanced truncation, particularly Lyapunov balanced truncation introduced by Moore, is widely used due to its stability preservation and a priori error bounds over the entire frequency range. However, in practical applications where input signals operate within a known finite frequency range \( \omega \in [\omega_1, \omega_2] \), the object

...


Summary chunk number 0
Neural network models have made significant advances in NLP tasks like speech recognition, sentiment analysis, text summarization, and machine translation. Although recurrent neural networks excel at modeling long-range word dependencies, current RNNLMs rely on a classical classification framework with two key limitations: (1) no natural metric is assumed on output classes, despite the existence of a well-defined metric space in language modeling via word embeddings; and (2) inputs and outputs are treated as isolated, whereas in language modeling they reside in the same space, leading to inefficiency in model complexity and information utilization. To address these issues, we propose a novel loss framework comprising two interlinked improvements: (1) augmenting cross-entropy loss with a term that minimizes KL-divergence between model predictions and a target distribution derived from word embedding similarities; and (2) theoretically analyzing this loss leads to

,0
0,INTRODUCTION
1,OUR APPROACH
2,MULTI-VIEW LEARNING OF ACOUSTIC WORD EMBEDDINGS
3,RECURRENT NEURAL NETWORK ARCHITECTURE
4,RELATED WORK
5,EXPERIMENTS AND RESULTS
6,DATA
7,MODEL DETAILS AND HYPERPARAMETER TUNING
8,EFFECTS OF DIFFERENT OBJECTIVES
9,Objective


Токенов в статье: 6253


  0%|          | 0/19 [00:00<?, ?it/s]


Summary chunk number 0
Word embeddings—continuous vector representations of words—are widely used in natural language processing. They can be learned via spectral methods or neural networks and are often composed to represent phrases, sentences, or documents. While most embeddings capture semantic meaning, few represent the acoustic characteristics of words. This work introduces a multi-view approach to learn embeddings that capture how words sound, using both acoustic and orthographic sequences. By jointly learning embeddings from matched and mismatched acoustic-orthographic pairs using contrastive losses, the method enables direct comparison between acoustic and orthographic embeddings. The approach improves performance in acoustic word discrimination and shows promising results in cross-view tasks and word similarity, offering a more effective alternative to sequence-based methods like dynamic time warping.

Summary chunk number 1
In this section, we first introduce our approach fo

In [ ]:
print("\n--- ROUGE Metrics (средние значения) ---")
for key in rouge_list[0].keys():
    avg = sum(r[key] for r in rouge_list) / len(rouge_list)
    print(f"{key}: {avg:.4f}")

avg_bert = sum(bert_list) / len(bert_list)
print(f"\n--- BERTScore F1 (среднее): {avg_bert:.4f} ---")

avg_sum = sum(l['for_summary'] for l in ldfacts_list) / len(ldfacts_list)
avg_abs = sum(l['for_abstract'] for l in ldfacts_list) / len(ldfacts_list)

print(f"\n--- LongDocFACTScore (средние) ---")
print(f"Для суммаризаций: {avg_sum:.4f}")
print(f"Для абстрактов: {avg_abs:.4f}")